In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf

In [2]:
df = pd.read_csv('https://github.com/clairett/pytorch-sentiment-classification/raw/master/data/SST2/train.tsv', delimiter='\t', header=None)

In [3]:
df.head()

,0,1
0,"a stirring , funny and finally transporting re...",1
1,apparently reassembled from the cutting room f...,0
2,they presume their audience wo n't sit still f...,0
3,this is a visually stunning rumination on love...,1
4,jonathan parker 's bartleby should have been t...,1


In [4]:
labels = df[1].values

In [5]:
labels.shape

(6920,)

In [6]:
texts = df[0].values.tolist()

In [7]:
from transformers import AutoTokenizer, TFGPT2ForSequenceClassification

In [8]:
tokenizer = AutoTokenizer.from_pretrained("microsoft/DialogRPT-updown")
model = TFGPT2ForSequenceClassification.from_pretrained("microsoft/DialogRPT-updown", from_pt=True)

C:\Users\yong\anaconda3\envs\textmining_cpu\lib\site-packages\huggingface_hub\file_download.py:133: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\yong\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


All PyTorch model weights were used when initializing TFGPT2ForSequenceClassification.

All the weights of TFGPT2ForSequenceClassification were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFGPT2ForSequenceClassification for predictions without further training.


In [9]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(texts, labels, test_size=0.2, random_state=0)

In [10]:
X_train_tokenized = tokenizer(X_train, return_tensors="np", max_length=30, padding='max_length', truncation=True)
X_test_tokenized = tokenizer(X_test, return_tensors="np", max_length=30, padding='max_length', truncation=True)

In [11]:
optimizer = tf.keras.optimizers.Adam(2e-5)
loss = tf.keras.losses.BinaryCrossentropy(from_logits=True)
model.compile(optimizer=optimizer, loss=loss, metrics=['accuracy'])

In [12]:
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import ModelCheckpoint
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=1)
checkpoint_filepath = "./checkpoints/checkpoint_gpt2"
mc = ModelCheckpoint(checkpoint_filepath, monitor='val_loss', mode='min', 
                     save_best_only=True, save_weights_only=True)

In [13]:
model.fit(dict(X_train_tokenized), y_train, epochs=10, batch_size=128, validation_split=0.1, callbacks=[es, mc])

Epoch 1/10
39/39 [==============================] - 1529s 36s/step - loss: 0.5928 - accuracy: 0.6381 - val_loss: 0.3817 - val_accuracy: 0.8177
Epoch 2/10
39/39 [==============================] - 1398s 36s/step - loss: 0.3385 - accuracy: 0.8497 - val_loss: 0.3115 - val_accuracy: 0.8700
Epoch 3/10
39/39 [==============================] - 1158s 30s/step - loss: 0.2258 - accuracy: 0.9115 - val_loss: 0.2801 - val_accuracy: 0.8845
Epoch 4/10
39/39 [==============================] - 804s 21s/step - loss: 0.1613 - accuracy: 0.9380 - val_loss: 0.2895 - val_accuracy: 0.8935
Epoch 4: early stopping


In [14]:
y_preds = model.predict(dict(X_test_tokenized))

44/44 [==============================] - 81s 2s/step


In [15]:
y_preds.logits

array([[-4.331848 ],
       [-5.092848 ],
       [ 0.3103965],
       ...,
       [ 5.0954065],
       [ 4.6633654],
       [ 2.372532 ]], dtype=float32)

In [16]:
prediction_probs=tf.nn.sigmoid(y_preds.logits).numpy()

In [17]:
y_predictions = [1 if x > 0.5 else 0 for x in prediction_probs ]

In [18]:
from sklearn.metrics import classification_report
print(classification_report(y_predictions, y_test))

              precision    recall  f1-score   support

           0       0.91      0.85      0.88       691
           1       0.86      0.91      0.89       693

    accuracy                           0.88      1384
   macro avg       0.88      0.88      0.88      1384
weighted avg       0.88      0.88      0.88      1384

